In [ ]:
import numpy as np
import pandas as pd

from my_utilities import *
from Make_QSO_altered_2 import add_errors

In [ ]:
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

Hubble_h = 0.73
Omega_M = 0.25
Omega_Lambda = 0.75
cosmo = FlatLambdaCDM(H0=Hubble_h*100, Om0=Omega_M)

In [ ]:
lc = np.load('/home/alberto/almacen/David_lightcone/LightCone_512_15sep2022.npy')

In [ ]:
def M_to_m(M, redshift, x, y, z):
    '''
    Computes the apparent magnitude from the absolute magnitude
    Inputs:
    M: Absolute magnitude
    redshift: Redshift of the source
    x, y, z: Coordinates in the Lightcone (pc)
    '''
    # Luminosity distance:
    dL = cosmo.luminosity_distance(redshift).to(u.pc).value.reshape(-1, 1)

    return M + 5 * np.log10(dL) - 5

In [ ]:
w_central = central_wavelength()

In [ ]:
M = lc['ObsMagDustLine']
M_noline = lc['ObsMagDustNoLine']
XX, YY, ZZ = lc['pos'].T
z = lc['redshift']
Rdisk = lc['Rdisk']
Rbulge = lc['Rbulge']
Mdisk = lc['DiskMass']
Mbulge = lc['BulgeMass']
OII_flux = lc['FluxDust_OII3729']
pm_mag = M_to_m(M, z, XX, YY, ZZ).T
pm_flx = mag_to_flux(pm_mag, w_central.reshape(-1, 1))
pm_mag_noline = M_to_m(M_noline, z, XX, YY, ZZ).T
pm_flx_noline = mag_to_flux(pm_mag_noline, w_central.reshape(-1, 1))

In [ ]:
import matplotlib.pyplot as plt
print(np.sum(pm_mag[-2] < 24))
plt.hist(pm_mag[-2], np.linspace(15, 27, 50))
plt.show()

In [ ]:
for survey_name in ['minijpasAEGIS001']:
    # pm_flx_noline, pm_err = add_errors(pm_flx_noline, apply_err=False, survey_name='minijpas')

    tcurves = np.load('../npy/tcurves.npy', allow_pickle=True).item()
    ftags = tcurves['tag']
    hdr = (ftags + [s + '_e' for s in ftags] + ['z', 'Rdisk', 'Rbulge', 'Mdisk', 'Mbulge'])

    data = np.vstack([pm_flx_noline, pm_flx, z, Rdisk, Rbulge, Mdisk, Mbulge]).T

    filename = f'/home/alberto/almacen/Source_cats/GAL_LC_{survey_name}_noline_0.csv'
    pd.DataFrame(data=data).to_csv(filename, header=hdr)